In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd

from preprocessing.preprocess_text import clean_text
from preprocessing.gen_text_embeddings import generate_text_embeddings
import torch
from pytorch_datasets.text_dataset import TextDataset
from torch.utils.data import DataLoader, random_split
from models.vae_text import TextVAE
from torch.optim import Adam

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/mehta.vats/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# Load and preprocess data

instagram_data = pd.read_csv("data/instagram_data.csv")
data = instagram_data.dropna(subset=["description"]).reset_index(drop=True)

print(f"Removed {len(instagram_data) - len(data)} rows due to N/A descriptions.")

post_descriptions = (
    data["description"]
    .apply(lambda text: clean_text(text) if type(text) == str else text)
    .tolist()
)

post_classes = data["Party"]

/tmp/ipykernel_7771/1832655542.py:3: DtypeWarning: Columns (24,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,62,63,65) have mixed types. Specify dtype option on import or set low_memory=False.
  instagram_data = pd.read_csv("data/instagram_data.csv")


Removed 7662 rows due to N/A descriptions.


In [3]:
# Generate Text Embeddings
text_embeddings, word_index_mapping = generate_text_embeddings(post_descriptions, 32)

In [4]:
# Create datasets
dataset = TextDataset(post_descriptions, word_index_mapping, post_classes)

train_dataset, test_dataset = random_split(dataset, [0.8, 0.2])
print(len(dataset))
print(len(train_dataset))
print(len(test_dataset))

373325
298660
74665


In [10]:
# Train model

num_epochs = 10
batch_size = 512
print(dataset.max_length)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

device = "cpu" # "cuda" if torch.cuda.is_available() else "cpu"

print(f"Using {device} device")

model = TextVAE(device, 32, 32, text_embeddings, dataset.padding_index, dataset.num_post_classes).to(device)
optimizer = Adam(model.parameters(), lr=1e-3)

for epoch in range(num_epochs):
    model.train()
    train_loss = 0

    for batch_idx, (text, party) in enumerate(train_dataloader):
        if batch_idx % 60 == 0:
            print("hang in there")
        text = text.to(device)

        decoded_text, mu, logvar, classifier_result = model(text)
        # print(party)
        # print(torch.argmax(classifier_result, dim=-1))
        
        optimizer.zero_grad()

        loss = model.loss_function(
            text, party, decoded_text, mu, logvar, classifier_result
        )
        loss.backward()

        train_loss += loss.item()
        optimizer.step()

    print(f"Train Epoch: {epoch} \tLoss: {loss.item():.6f}")


311
Using cpu device
208232
208231
hang in there


RuntimeError: [enforce fail at alloc_cpu.cpp:75] err == 0. DefaultCPUAllocator: can't allocate memory: you tried to allocate 265257582592 bytes. Error code 12 (Cannot allocate memory)